In [2]:
import pandas
import pickle as pickle
import backtest
import numpy
import math

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%config InlineBackend.figure_format = 'svg'

In [3]:
# index data
chsashr_pi = pandas.read_csv('csvs/index_chsashr.csv')
chsashr_pi['dates'] = chsashr_pi['Unnamed: 0']
chsashr_pi = chsashr_pi.set_index('dates')
chsashr_pi = chsashr_pi.drop('Unnamed: 0', 1)
chsashr_pi['index_returns'] = chsashr_pi['PI'].pct_change()
chsashr_pi['index_returns'] = chsashr_pi['index_returns'] + 1

chzashr_pi = pandas.read_csv('csvs/index_chzashr.csv')
chzashr_pi['dates'] = chzashr_pi['Unnamed: 0']
chzashr_pi = chzashr_pi.set_index('dates')
chzashr_pi = chzashr_pi.drop('Unnamed: 0', 1)
chzashr_pi['index_returns'] = chzashr_pi['PI'].pct_change()
chzashr_pi['index_returns'] = chzashr_pi['index_returns'] + 1

In [4]:
# df_sh = pickle.load(open('temp/df_sh.pkl', 'rb'))
# date_sh = pickle.load(open('temp/date_sh.pkl', 'rb'))
# dates = pickle.load(open('temp/dates.pkl', 'rb'))
# universes = pickle.load(open('temp/universes_sh.pkl', 'rb'))
# signals_value_sh = pickle.load(open('temp/signals_value_sh.pkl', 'rb'))

df_sh = pickle.load(open('temp_momentum/df_sh.pkl', 'rb'))
date_sh = pickle.load(open('temp_momentum/date_sh.pkl', 'rb'))
dates = pickle.load(open('temp_momentum/dates.pkl', 'rb'))
universes = pickle.load(open('temp_momentum/universes_sh.pkl', 'rb'))
signals_value_sh = pickle.load(open('temp_momentum/signals_momentum_sh.pkl', 'rb'))

In [5]:
period_returns = []
dates_used = []

holding_period = 4

for i in range(0, len(dates) - holding_period, holding_period):
    date_t0 = dates[i]
    date_t1 = dates[i + holding_period]
    period_return = backtest.period_return(signals_value_sh[i], df_sh, date_t0, date_t1)
    period_returns.append(period_return)
    dates_used.append(date_t0)

pickle.dump(period_returns, open('temp/period_returns_sh.pkl', 'wb'))

In [6]:
period_returns_sh = pickle.load(open('temp/period_returns_sh.pkl', 'rb'))
plot_data = pandas.DataFrame({'returns': period_returns_sh, 'dates': dates_used})
plot_data['dates'] = pandas.to_datetime(plot_data['dates'])
plot_data = plot_data.set_index('dates')
plot_data['returns'] = plot_data['returns'] + 1
plot_data['returns_cumprod'] = plot_data['returns'].cumprod()
plot_data = plot_data.merge(chsashr_pi, left_index=True, right_index=True)
plot_data['index_returns_cumprod'] = plot_data['index_returns'].cumprod()
plot_data['returns_std'] = plot_data['returns_cumprod'].rolling(window=52, min_periods=52).std()
plot_data['returns_annualized'] = (plot_data['returns_cumprod'] - plot_data['returns_cumprod'].shift(52)) / plot_data['returns_cumprod'].shift(52)
plot_data['index_returns_std'] = plot_data['index_returns_cumprod'].rolling(window=52, min_periods=52).std()
plot_data['index_returns_annualized'] = (plot_data['index_returns_cumprod'] - plot_data['index_returns_cumprod'].shift(52)) / plot_data['index_returns_cumprod'].shift(52)

# plot_data
plot_data[['index_returns_cumprod', 'returns_cumprod']].plot()

In [7]:
plot_data

returns  returns_cumprod        PI  index_returns  \
2000-01-03  1.111726         1.111726  1451.900       1.015698   
2000-01-31  1.020441         1.134451  1631.524       1.038764   
2000-02-28  0.981828         1.113836  1814.783       1.016995   
2000-03-27  0.917199         1.021610  1889.534       1.044498   
2000-04-24  1.030117         1.052378  1954.889       1.023339   
2000-05-22  0.924735         0.973171  1941.130       1.070551   
2000-06-19  0.964972         0.939083  2034.282       1.000981   
2000-07-17  1.014054         0.952280  2105.201       1.017316   
2000-08-14  1.000712         0.952959  2200.368       1.017257   
2000-09-11  0.992751         0.946051  2089.886       0.980332   
2000-10-09  0.975220         0.922608  2032.076       1.002819   
2000-11-06  0.985621         0.909342  2109.290       1.008608   
2000-12-04  0.980777         0.891862  2216.103       1.020205   
2001-01-01  0.988137         0.881283  2192.378       1.001378   
2001-01-29  1.010594         0.890619  2185.196       1.000000   
2001-02-26  1.011364         0.900740  2067.197       0.993387   
2001-03-26  1.005016         0.905259  2192.069       1.024878   
2001-04-23  1.016740         0.920412  2247.221       0.990878   
2001-05-21  1.010113         0.929721  2305.680       1.014719   
2001-06-18  1.002566         0.932107  2261.038       0.978922   
2001-07-16  1.039396         0.968828  2235.154       0.988463   
2001-08-13  0.990097         0.959234  2044.251       1.037030   
2001-09-10  0.995036         0.954472  1938.065       1.020855   
2001-10-08  1.001288         0.955702  1803.659       0.977969   
2001-11-05  0.967877         0.925001  1752.199       0.987585   
2001-12-03  1.029549         0.952334  1837.553       1.032422   
2001-12-31  1.038772         0.989258  1712.541       1.024645   
2002-01-28  0.957119         0.946837  1418.281       0.997131   
2002-02-25  0.987008         0.934536  1595.979       1.015826   
2002-03-25  1.000689         0.935181  1748.447       1.032200   
...              ...              ...       ...            ...   
2013-12-16  1.041329         1.440366  2261.862       0.965414   
2014-01-13  1.099916         1.584281  2103.066       0.982271   
2014-02-10  0.977832         1.549161  2183.225       1.026136   
2014-03-10  0.951158         1.473497  2092.751       0.963263   
2014-04-07  1.000784         1.474653  2155.532       1.012564   
2014-05-05  1.000543         1.475453  2122.481       1.011880   
2014-06-02  1.015868         1.498866  2135.087       0.999006   
2014-06-30  0.926778         1.389116  2144.746       1.011852   
2014-07-28  1.039166         1.443523  2280.536       1.060218   
2014-08-25  0.997965         1.440586  2333.800       0.995476   
2014-09-22  0.988901         1.424597  2397.161       0.978926   
2014-10-20  0.979314         1.395128  2467.345       0.996070   
2014-11-17  0.946775         1.320873  2590.627       1.000113   
2014-12-15  0.899111         1.187612  3093.680       0.977680   
2015-01-12  0.995470         1.182231  3383.836       0.963768   
2015-02-09  0.975270         1.152994  3242.713       0.989360   
2015-03-09  1.018167         1.173941  3460.377       0.989814   
2015-04-06  1.086527         1.275519  4050.095       1.020477   
2015-05-04  0.952003         1.214297  4694.551       0.989558   
2015-06-01  1.002633         1.217494  5056.188       1.002926   
2015-06-29  0.974712         1.186706  4245.469       0.905302   
2015-07-27  0.983282         1.166866  3903.456       0.933252   
2015-08-24  0.937832         1.094324  3362.830       0.803710   
2015-09-21  1.002756         1.097340  3306.421       1.013193   
2015-10-19  0.984961         1.080837  3547.078       1.030107   
2015-11-16  0.983445         1.062945  3777.327       0.988974   
2015-12-14  0.956444         1.016648  3685.935       0.995301   
2016-01-11  1.023784         1.040827  3157.712       0.915271   
2016-02-08  0.920315         0.957889  2891.622      